In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## bUILT-iN pACKAGES
import sys, os, time, bz2, zlib, pickle, math, json, csv
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
%matplotlib inline
from IPython.display import display

## lAST sUMMER
from romtoslp import *
from sentences import *
from DCS import *
import MatDB
from heap_n_PrimMST import *
from ECL_MST import *

## lAST yEAR
# from word_definite import *
from nnet import *
# from heap_n_PrimMST import *
# from word_definite import *

In [3]:
# from MatDB import *
import word_definite as WD
from heap_n_PrimMST import *
from nnet import *

In [ ]:
import winsound
def playBeep():
    for i in range(3):
        winsound.Beep(2200, 300)
        winsound.Beep(2600, 300)

In [4]:
loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'), encoding=u'utf-8')
loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'), encoding=u'utf-8')

In [6]:
# dataset_4k_1k = pickle.load(open('../SmallDataset_4K_1K.p', 'rb'))
# TrainFiles = dataset_4k_1k['TrainFiles']
# TestFiles = dataset_4k_1k['TestFiles']

# dataset_6k_3k = pickle.load(open('../SmallDataset_6K_3K.p', 'rb'))
# TrainFiles_2 = dataset_6k_3k['TrainFiles']
# TestFiles_2 = dataset_6k_3k['TestFiles']

In [7]:
# matDB = MatDB.MatDB()

In [5]:
# Create Training File List
excluded_files = []
with open('inputs/Baseline4_advSample.csv', 'r') as f_handle:
    opener = csv.reader(f_handle)
    for line in opener:
        excluded_files.append(line[1].replace('.p', '.ds.bz2'))

bz2_input_folder = '../NewData/skt_dcs_DS.bz2_1L_bigram_10K/'
# bz2_input_folder = '/home/rs/15CS91R05/vishnu/Data/skt_dcs_DS.bz2_compat_10k_check_again/'
all_files = []
skipped = 0
for f in os.listdir(bz2_input_folder):
    if '.ds.bz2' in f:
        if f in excluded_files:
            skipped += 1
            continue
        all_files.append(f)

print(skipped, 'files will not be used for training')
print('Size of training set:', len(all_files))

TrainFiles = all_files        

957 files will not be used for training
Size of training set: 8219


In [6]:
def open_dsbz2(filename):
    with bz2.BZ2File(filename, 'r') as f:
        loader = pickle.load(f)
    
    conflicts_Dict_correct = loader['conflicts_Dict_correct']
    nodelist_to_correct_mapping = loader['nodelist_to_correct_mapping']
    nodelist_correct = loader['nodelist_correct']
    featVMat_correct = loader['featVMat_correct']
    featVMat = loader['featVMat']
    conflicts_Dict = loader['conflicts_Dict']
    nodelist = loader['nodelist']
    
    return (nodelist_correct, conflicts_Dict_correct, featVMat_correct, nodelist_to_correct_mapping,\
            nodelist, conflicts_Dict, featVMat)


In [7]:
"""
################################################################################################
######################  CREATE SEVERAL DATA STRUCTURES FROM SENTENCE/DCS  ######################
###########################  NODELIST, ADJACENCY LIST, GRAPH, HEAP #############################
################################################################################################
"""
def GetTrainingKit(sentenceObj, dcsObj):
    nodelist = GetNodes(sentenceObj)
    
    # Nodelist with only the correct_nodes
    nodelist2 = GetNodes(sentenceObj)
    nodelist2_to_correct_mapping = {}
    nodelist_correct = []
    search_key = 0
    first_key = 0
    for chunk_id in range(len(dcsObj.lemmas)):
        while nodelist2[first_key].chunk_id != chunk_id:
            first_key += 1
        for j in range(len(dcsObj.lemmas[chunk_id])):
            search_key = first_key
            while (nodelist2[search_key].lemma != rom_slp(dcsObj.lemmas[chunk_id][j])) or (nodelist2[search_key].cng != dcsObj.cng[chunk_id][j]):
                search_key += 1
                if search_key >= len(nodelist2) or nodelist2[search_key].chunk_id > chunk_id:
                    break
    #         print((rom_slp(dcsObj.lemmas[chunk_id][j]), dcsObj.cng[chunk_id][j]))
    #         print(nodelist[search_key])
            nodelist2_to_correct_mapping[len(nodelist_correct)] = search_key
            nodelist_correct.append(nodelist2[search_key])
    return (nodelist, nodelist_correct, nodelist2_to_correct_mapping)
    

def GetGraph(nodelist, neuralnet):
    if not neuralnet.outer_relu:
        conflicts_Dict = Get_Conflicts(nodelist)

        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)

        (WScalarMat, SigmoidGateOutput) = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        return (conflicts_Dict, featVMat, WScalarMat, SigmoidGateOutput)
    else:
        conflicts_Dict = Get_Conflicts(nodelist)

        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)

        WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        return (conflicts_Dict, featVMat, WScalarMat)

In [8]:
trainingStatus = defaultdict(lambda: bool(False))

In [9]:
"""
################################################################################################
##############################  TRAIN FUNCTION  ################################################
################################################################################################
"""

def train_generator(loaded_SKT, loaded_DCS, bz2_input_folder, n_trainset = -1, iterationPerBatch = 10, filePerBatch = 20, _debug = True):
    # Train
    if n_trainset == -1:
        n_trainset = len(TrainFiles)
        totalBatchToTrain = math.ceil(n_trainset/filePerBatch)
    else:
        totalBatchToTrain = math.ceil(n_trainset/filePerBatch)
    
    register_nnet(trainer.neuralnet, bz2_input_folder)
    for iterout in range(totalBatchToTrain):
        # Add timer
        startT = time.time()

        # Change current batch
        if(iterout % 50 == 0):
            trainer.Save(p_name.replace('.p', '_i{}.p'.format(iterout)))
        else:
            trainer.Save(p_name)
        print('Batch: ', iterout)
        files_for_batch = TrainFiles[iterout*filePerBatch:(iterout + 1)*filePerBatch]
        print(files_for_batch)
        # trainer.Load('outputs/neuralnet_trained.p')
        try:
            # Run few times on same set of files
            for iterin in range(iterationPerBatch):
                print('ITERATION IN', iterin)        
                for fn in files_for_batch:
                    trainFileName = fn.replace('.ds.bz2', '.p2')
                    sentenceObj = loaded_SKT[trainFileName]
                    dcsObj = loaded_DCS[trainFileName]
                    if trainingStatus[sentenceObj.sent_id]:
                        continue
                    # trainer.Save('outputs/saved_trainer.p')
                    try:
                        trainer.Train(sentenceObj, dcsObj, bz2_input_folder, _debug)
                    except (IndexError, KeyError) as e:
                        print('\x1b[31mFailed: {} \x1b[0m'.format(sentenceObj.sent_id))
                sys.stdout.flush() # Flush IO buffer 
            finishT = time.time()
            print('Avg. time taken by 1 file(1 iteration): {:.3f}'.format((finishT - startT)/(iterationPerBatch*filePerBatch)))
        except KeyboardInterrupt:
            print('Training paused')
            trainer.Save(p_name)
            yield None
    trainer.Save(p_name)
                
def test(loaded_SKT, loaded_DCS, n_testSet = -1, _testFiles = None, n_checkpt = 100):
    total_lemma = 0;
    correct_lemma = 0;

    total_word = 0;
    total_output_nodes = 0
    correct_word = 0;
    file_counter = 0
    if _testFiles is None:
        if n_testSet == -1:
            _testFiles = TestFiles
        else:
            _testFiles = TestFiles[0:n_testSet]
    else:
        if n_testSet == -1:
            _testFiles = _testFiles
        else:
            _testFiles = _testFiles[0:n_testSet]
            
    recalls = []
    recalls_of_word = []
    precisions = []
    precisions_of_words = []
    for fn in _testFiles:
        if file_counter % n_checkpt == 0:
            print(file_counter,' Checkpoint... ')
            sys.stdout.flush() # Flush IO buffer 
        
        file_counter += 1
        
        testFileName = fn.replace('.ds.bz2', '.p2')
        sentenceObj = loaded_SKT[testFileName]
        dcsObj = loaded_DCS[testFileName]
        
        try:
            (word_match, lemma_match, n_dcsWords, n_output_nodes) = trainer.Test(sentenceObj, dcsObj)
            
            recalls.append(lemma_match/n_dcsWords)
            recalls_of_word.append(word_match/n_dcsWords)
            
            precisions.append(lemma_match/n_output_nodes)
            precisions_of_words.append(word_match/n_output_nodes)
            
            total_lemma += n_dcsWords
            total_word += n_dcsWords
            
            total_output_nodes += n_output_nodes            
            
            correct_lemma += lemma_match
            correct_word += word_match
        except (IndexError, KeyError) as e:
            print('Failed!')        

    print('Avg. Micro Recall of Lemmas: {}'.format(np.mean(np.array(recalls))))
    print('Avg. Micro Recall of Words: {}'.format(np.mean(np.array(recalls_of_word))))
    print('Avg. Micro Precision of Lemmas: {}'.format(np.mean(np.array(precisions))))
    print('Avg. Micro Precision of Words: {}'.format(np.mean(np.array(precisions_of_words))))
    
    return (recalls, recalls_of_word, precisions, precisions_of_words)
    

In [10]:
# NOW-OLD FUNCTION
# def GetLoss(_mst_adj_graph, _mask_de_correct_edges, _negLogLikelies):
#     _negLogLikelies = _negLogLikelies.copy()
#     _negLogLikelies[~_mst_adj_graph] = 0
#     _negLogLikelies[~_mask_de_correct_edges] *= -1 # BAKA!!! Check before you try to fix this again
#     return np.sum(_negLogLikelies)    

# NEW FUNCTION
def GetLoss(_mst_adj_graph, _mask_de_correct_edges, _WScalarMat):
    _WScalarMat = _WScalarMat.copy()
    _WScalarMat[_mst_adj_graph&(~_mask_de_correct_edges)] *= -1 # BAKA!!! Check before you try to fix this again
    _WScalarMat[~_mst_adj_graph] = 0
    return np.sum(_WScalarMat)

In [11]:
"""
################################################################################################
##############################  GET A FILENAME TO SAVE WEIGHTS  ################################
################################################################################################
"""
import time
st = str(int((time.time() * 1e6) % 1e13))
log_name = 'logs/train_nnet_t{}.out'.format(st)
odir = 'outputs/train_t{}'.format(st)
p_name = 'outputs/train_t{}/nnet.p'.format(st)
print('nEURAL nET wILL bE sAVED hERE: ', p_name)

nEURAL nET wILL bE sAVED hERE:  outputs/train_t1993934581183/nnet.p


In [12]:
def register_nnet(nnet, bz2_input_folder):
    if not os.path.isdir(odir):
        os.mkdir(odir)
    if not os.path.isfile('outputs/nnet_LOGS.csv'):
        with open('outputs/nnet_LOGS.csv', 'a') as fh:
            csv_r = csv.writer(fh)
            csv_r.writerow(['odir', 'p_name', 'hidden_layer_size', '_edge_vector_dim'])
    with open('outputs/nnet_LOGS.csv', 'a') as fh:
        csv_r = csv.writer(fh)
        if nnet.version == 'h1':
            csv_r.writerow([odir, p_name, nnet.n, nnet.d, bz2_input_folder])
        elif nnet.version == 'h2':
            csv_r.writerow([odir, p_name, nnet.h1, nnet.h2, nnet.d, bz2_input_folder])

In [60]:
class Trainer:
    def __init__(self, modelFile = None):
        if modelFile is None:
            self._edge_vector_dim = 1500
            self.hidden_layer_size = 60
            # self._edge_vector_dim = WD._edge_vector_dim
            # self._full_cnglist = list(WD.mat_cngCount_1D)
            self.neuralnet = NN(self._edge_vector_dim, self.hidden_layer_size, outer_relu=True)
            
            # DeepR Network
            '''
            self.hidden_layer_size = 150
            self.hidden_layer_size2 = 20
            self.neuralnet = NN_2(self._edge_vector_dim, self.hidden_layer_size,\
                                  hidden_layer_2_size = self.hidden_layer_size2, outer_relu=True)
            self.history = defaultdict(lambda: list())
            #'''
        else:
            loader = pickle.load(open(filename, 'rb'))
            
            self.neuralnet.n = loader['n']
            self.neuralnet.d = loader['d']

            self.neuralnet = NN(self._edge_vector_dim, self.hidden_layer_size, outer_relu=True)

            self.neuralnet.U = loader['U']
            self.neuralnet.W = loader['W']
            self.neuralnet.B1 = loader['B1']
            self.neuralnet.B2 = loader['B2']
            
            self.history = defaultdict(lambda: list())
            
        # SET LEARNING RATES
        if self.neuralnet.version == 'h1':
            self.neuralnet.etaW = 3e-3
            self.neuralnet.etaB1 = 1e-3

            self.neuralnet.etaU = 1e-3
            self.neuralnet.etaB2 = 1e-3
        elif self.neuralnet.version == 'h2':
            self.neuralnet.etaW1 = 3e-4
            self.neuralnet.etaB1 = 1e-4

            self.neuralnet.etaW2 = 1e-4
            self.neuralnet.etaB2 = 1e-4
            
            self.neuralnet.etaU = 1e-4
            self.neuralnet.etaB3 = 1e-4
            
            
    def Reset(self):
        self.neuralnet = NN(self._edge_vector_dim, self.hidden_layer_size)
        self.history = defaultdict(lambda: list())
        
    def Save(self, filename):
        print('Weights Saved: ', filename)
        if self.neuralnet.version == 'h1':
            pickle.dump({
                    'U': self.neuralnet.U,
                    'W': self.neuralnet.W,
                    'n': self.neuralnet.n,
                    'd': self.neuralnet.d,
                    'B1': self.neuralnet.B1,
                    'B2': self.neuralnet.B2,
                    'version': self.neuralnet.version
                }, open(filename, 'wb'))
            return
        elif self.neuralnet.version == 'h2':
            pickle.dump({
                    'U': self.neuralnet.U,
                    'B3': self.neuralnet.B3,
                    'W2': self.neuralnet.W2,
                    'B2': self.neuralnet.B2,
                    'W1': self.neuralnet.W1,
                    'B1': self.neuralnet.B1,
                    'h1': self.neuralnet.h1,
                    'h2': self.neuralnet.h2,
                    'd': self.neuralnet.d,
                    'version': self.neuralnet.version
                }, open(filename, 'wb'))
            return
        
    
    def Load(self, filename):
        loader = pickle.load(open(filename, 'rb'))
        if 'version' not in loader: # means 1 hidden layer
            self.neuralnet = NN(self._edge_vector_dim, self.hidden_layer_size, outer_relu=True)
            self.neuralnet.U = loader['U']
            self.neuralnet.W = loader['W']
            self.neuralnet.B1 = loader['B1']
            self.neuralnet.B2 = loader['B2']
            self.neuralnet.hidden_layer_size = loader['n']
            self.neuralnet._edge_vector_dim = loader['d']
        else:
            if loader['version'] == 'h1':
                self.neuralnet = NN(self._edge_vector_dim, self.hidden_layer_size, outer_relu=True)
                self.neuralnet.U = loader['U']
                self.neuralnet.W = loader['W']
                self.neuralnet.B1 = loader['B1']
                self.neuralnet.B2 = loader['B2']
                self.neuralnet.hidden_layer_size = loader['n']
                self.neuralnet._edge_vector_dim = loader['d']
            elif loader['version'] == 'h2':
                self.neuralnet = NN_2(self._edge_vector_dim, self.hidden_layer_size, outer_relu=True)
                
                self.neuralnet.U = loader['U']
                self.neuralnet.B3 = loader['B3']
                
                self.neuralnet.W2 = loader['W2']
                self.neuralnet.B2 = loader['B2']
                
                self.neuralnet.W1 = loader['W1']
                self.neuralnet.B1 = loader['B1']
                
                self.neuralnet.h1 = loader['h1']
                self.neuralnet.h2 = loader['h2']
                self.neuralnet.d = loader['d']
    def CalculateLoss_n_Grads(self, WScalarMat, min_st_adj_worst, max_st_adj_gold, loss_type = 0, min_marginalized_energy = None):
        doBpp = True
        
        # Claculate the enrgies
        etg = np.sum(WScalarMat[max_st_adj_gold])
        etq = np.sum(WScalarMat[min_st_adj_worst])
        
        if loss_type == 0:
            # Variable Hinge Loss - CHECKED
            L = etg - min_marginalized_energy
            if L > 0:
                dLdOut = np.zeros_like(WScalarMat)
                dLdOut[max_st_adj_gold&(~min_st_adj_worst)] = 1
                dLdOut[(~max_st_adj_gold)&min_st_adj_worst] = -1
            else:
                doBpp = False
        elif loss_type == 1:
            # LOg Loss
            a = etg - etq
            b = np.exp(a)
            L = np.log(1 + b)
            
            dLdOut = np.zeros_like(WScalarMat)
            dLdOut[max_st_adj_gold&(~min_st_adj_worst)] = 1
            dLdOut[(~max_st_adj_gold)&min_st_adj_worst] = -1
            
            dLdOut *= (b/(1 + b))
        elif loss_type == 2:
            # Square exponential loss
            gamma = 1
            b = np.exp(-etq)
            
            L = etg**2 + gamma*b
            
            dLdOut = np.zeros_like(WScalarMat)
            dLdOut[max_st_adj_gold&(~min_st_adj_worst)] = 2*etg
            dLdOut[(~max_st_adj_gold)&min_st_adj_worst] = -gamma*b
            pass
        return (L, dLdOut, doBpp)
    def Test(self, sentenceObj, dcsObj, dsbz2_name):
        neuralnet = self.neuralnet
        minScore = np.inf
        minMst = None
        
        # dsbz2_name = sentenceObj.sent_id + '.ds.bz2'
        (nodelist_correct, conflicts_Dict_correct, featVMat_correct, nodelist_to_correct_mapping,\
            nodelist, conflicts_Dict, featVMat) = open_dsbz2(dsbz2_name)
        
        # if len(nodelist) > 50:
        #     return None

        if not self.neuralnet.outer_relu:
            (WScalarMat, SigmoidGateOutput) = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        else:
            WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        
        # print('NeuralNet Time: ', time.time() - startT)
        # startT = time.time()
        
        # Get all MST
        for source in range(len(nodelist)):
            (mst_nodes, mst_adj_graph, _) = MST(nodelist, WScalarMat, conflicts_Dict, source)
            # print('.', end = '')
            score = GetMSTWeight(mst_adj_graph, WScalarMat)
            if(score < minScore):
                minScore = score
                minMst = mst_nodes
        dcsLemmas = [[rom_slp(l) for l in arr]for arr in dcsObj.lemmas]
        word_match = 0
        lemma_match = 0
        n_output_nodes = 0
        for chunk_id, wdSplit in minMst.items():
            for wd in wdSplit:
                n_output_nodes += 1
                # Match lemma
                search_result = [i for i, j in enumerate(dcsLemmas[chunk_id]) if j == wd.lemma]
                if len(search_result) > 0:
                    lemma_match += 1
                # Match CNG
                for i in search_result:
                    if(dcsObj.cng[chunk_id][i] == str(wd.cng)):
                        word_match += 1
                        # print(wd.lemma, wd.cng)
                        break
        dcsLemmas = [l for arr in dcsObj.lemmas for l in arr]
        
        # print('All MST Time: ', time.time() - startT)
        # print('Node Count: ', len(nodelist))
#         print('\nFull Match: {}, Partial Match: {}, OutOf {}, NodeCount: {}, '.\
#               format(word_match, lemma_match, len(dcsLemmas), len(nodelist)))
        return (word_match, lemma_match, len(dcsLemmas), n_output_nodes)
    
    def Train(self, sentenceObj, dcsObj, bz2_input_folder, _debug = True):
        # Hyperparameter for hinge loss: m
        m_hinge_param = 14
        
        dsbz2_name = sentenceObj.sent_id + '.ds.bz2'
        (nodelist_correct, conflicts_Dict_correct, featVMat_correct, nodelist_to_correct_mapping,\
            nodelist, conflicts_Dict, featVMat) = open_dsbz2(bz2_input_folder + dsbz2_name)
        # Train for large graphs separately
#         if len(nodelist) < 40:
#             return
        
        """ FORM MAXIMUM(ENERGY) SPANNING TREE OF THE GOLDEN GRAPH : WORST GOLD STRUCTURE """
        WScalarMat_correct = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat_correct, nodelist_correct,\
                                                                      conflicts_Dict_correct, self.neuralnet)
        source = 0
        """ Find the max spanning tree : negative Weight matrix passed """
#         (max_st_gold_ndict, max_st_adj_gold_small, _) =\
#             MST(nodelist_correct, -WScalarMat_correct, conflicts_Dict_correct, source)
        (max_st_gold_ndict, max_st_adj_gold_small, _) =\
            MST(nodelist_correct, -WScalarMat_correct, conflicts_Dict_correct, source)
        energy_gold_max_ST = np.sum(WScalarMat_correct[max_st_adj_gold_small])
        
        """ Convert correct spanning tree graph adj matrix to full marix dimensions """
        """ Create full-size adjacency matrix for correct_mst_small """
        nodelen = len(nodelist)
        max_st_adj_gold = np.ndarray((nodelen, nodelen), np.bool)*False # T_STAR
        for i in range(max_st_adj_gold_small.shape[0]):
            for j in range(max_st_adj_gold_small.shape[1]):
                max_st_adj_gold[nodelist_to_correct_mapping[i], nodelist_to_correct_mapping[j]] =\
                    max_st_adj_gold_small[i, j]
        
        """ Delta(Margin) Function : MASK FOR WHICH NODES IN NODELIST BELONG TO DCS """
        gold_nodes_mask = np.array([False]*len(nodelist))
        gold_nodes_mask[list(nodelist_to_correct_mapping.values())] = True
        margin_f = lambda nodes_mask: np.sum(nodes_mask&(~gold_nodes_mask))**2
        
        """ FOR ALL POSSIBLE MST FROM THE COMPLETE GRAPH """
        WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, self.neuralnet)

        """ For each node - Find MST with that source"""
        min_STx = None # Min Energy spanning tree with worst margin with gold_STx
        min_marginalized_energy = np.inf
        
        # Generate random set of nodes from which mSTs are to be considered
        n_nodes = len(nodelist)
        selection_prob = 0.4
        select_flag = np.random.rand(n_nodes) < selection_prob
        # Fix if all zeros
        if np.sum(select_flag) == 0:
            select_flag[np.random.randint(n_nodes)] = 1
        
        best_node_diff = np.Inf
        best_energy = np.Inf
        for source in range(len(nodelist)):
            (mst_nodes, mst_adj_graph, mst_nodes_bool) = MST(nodelist, WScalarMat, conflicts_Dict, source) # T_X
            # Calculate energy of spanning tree
            en_st = np.sum(WScalarMat[mst_adj_graph])
            
            # Pick up the node_diff with lowest energy
            delta_st = margin_f(mst_nodes_bool)
            if _debug:
                if best_energy > en_st:
                    best_node_diff = delta_st
                    best_energy = en_st
                
            # Minimum marginalized energy calculation
            marginalized_en = en_st - delta_st
            # Minimum marginalized spanning tree : Randomization applied
            # if marginalized_en < min_marginalized_energy and select_flag[source]:
            if marginalized_en < min_marginalized_energy:
                min_marginalized_energy = marginalized_en
                min_STx = mst_adj_graph
            # Energy diff should all be negative
            if _debug:
                print('Source: [{}], Node_Diff:{}, Max_Gold_En: {:.3f}, Energy: {:.3f}'.\
                      format(source, np.sum((~gold_nodes_mask)&mst_nodes_bool), energy_gold_max_ST,  np.sum(WScalarMat[mst_adj_graph])))

        if _debug:
            print('Best Node diff: {} with EN: {}'.format(np.sqrt(best_node_diff), best_energy))
        """ Gradient Descent """
        # LOSS TYPES -> hinge(0), log-loss(1), square-exponential(2)
        Total_Loss, dLdOut, doBpp = self.CalculateLoss_n_Grads(WScalarMat, min_STx, max_st_adj_gold,\
                                                                 loss_type = 1, min_marginalized_energy = min_marginalized_energy)
        if doBpp:
            if _debug:
                print('{}. '.format(sentenceObj.sent_id), end = '')
            self.neuralnet.Back_Prop(dLdOut, len(nodelist), featVMat, _debug)
        else:
            trainingStatus[sentenceObj.sent_id] = True
        if _debug:
            print("\nFileKey: %s, Loss: %6.3f" % (sentenceObj.sent_id, Total_Loss))
    

In [61]:
# trainer.neuralnet.U

In [62]:
trainer = None
def InitModule():
    global trainer
    trainer = Trainer()
InitModule()
trainingStatus = defaultdict(lambda: bool(False))
# trainer.Load('outputs/train_nnet_t526539685574.p')

In [63]:
"""
################################################################################################
################################################################################################
################################################################################################
"""
# if __name__ == '__main__':
# loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'))
# loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'))
# main(loaded_SKT, loaded_DCS)
np.set_printoptions(suppress=False)

ft = '433682.ds.bz2'
fn = ft.replace('.ds.bz2', '.p2')

print(fn)
for _ in range(10):
    trainer.Train(loaded_SKT[fn], loaded_DCS[fn], bz2_input_folder, _debug = True)

# # fn = TestFiles[1].replace('.ds.bz2', '.p2')
# trainer.Test(loaded_SKT[fn], loaded_DCS[fn])
# print ("Not Implemented")

433682.p2
Source: [0], Node_Diff:8, Max_Gold_En: 1.834, Energy: -0.859
Source: [1], Node_Diff:8, Max_Gold_En: 1.834, Energy: -0.859
Source: [2], Node_Diff:8, Max_Gold_En: 1.834, Energy: -0.859
Source: [3], Node_Diff:8, Max_Gold_En: 1.834, Energy: -0.859
Source: [4], Node_Diff:7, Max_Gold_En: 1.834, Energy: -1.020
Source: [5], Node_Diff:8, Max_Gold_En: 1.834, Energy: -0.859
Source: [6], Node_Diff:8, Max_Gold_En: 1.834, Energy: -0.859
Source: [7], Node_Diff:8, Max_Gold_En: 1.834, Energy: -0.859
Source: [8], Node_Diff:8, Max_Gold_En: 1.834, Energy: -1.518
Source: [9], Node_Diff:8, Max_Gold_En: 1.834, Energy: -0.897
Source: [10], Node_Diff:8, Max_Gold_En: 1.834, Energy: -0.800
Source: [11], Node_Diff:8, Max_Gold_En: 1.834, Energy: -0.803
Source: [12], Node_Diff:8, Max_Gold_En: 1.834, Energy: -0.797
Source: [13], Node_Diff:8, Max_Gold_En: 1.834, Energy: -0.801
Source: [14], Node_Diff:8, Max_Gold_En: 1.834, Energy: -1.198
Source: [15], Node_Diff:6, Max_Gold_En: 1.834, Energy: -2.495
Source: 

Source: [0], Node_Diff:7, Max_Gold_En: 162.967, Energy: 233.174
Source: [1], Node_Diff:7, Max_Gold_En: 162.967, Energy: 233.092
Source: [2], Node_Diff:7, Max_Gold_En: 162.967, Energy: 230.246
Source: [3], Node_Diff:7, Max_Gold_En: 162.967, Energy: 231.601
Source: [4], Node_Diff:7, Max_Gold_En: 162.967, Energy: 233.174
Source: [5], Node_Diff:7, Max_Gold_En: 162.967, Energy: 232.734
Source: [6], Node_Diff:7, Max_Gold_En: 162.967, Energy: 233.174
Source: [7], Node_Diff:7, Max_Gold_En: 162.967, Energy: 232.734
Source: [8], Node_Diff:7, Max_Gold_En: 162.967, Energy: 236.523
Source: [9], Node_Diff:7, Max_Gold_En: 162.967, Energy: 230.162
Source: [10], Node_Diff:7, Max_Gold_En: 162.967, Energy: 239.140
Source: [11], Node_Diff:7, Max_Gold_En: 162.967, Energy: 239.143
Source: [12], Node_Diff:7, Max_Gold_En: 162.967, Energy: 239.175
Source: [13], Node_Diff:7, Max_Gold_En: 162.967, Energy: 239.186
Source: [14], Node_Diff:7, Max_Gold_En: 162.967, Energy: 236.886
Source: [15], Node_Diff:7, Max_Gold

Source: [0], Node_Diff:7, Max_Gold_En: 163.433, Energy: 234.389
Source: [1], Node_Diff:7, Max_Gold_En: 163.433, Energy: 234.269
Source: [2], Node_Diff:7, Max_Gold_En: 163.433, Energy: 231.456
Source: [3], Node_Diff:7, Max_Gold_En: 163.433, Energy: 232.817
Source: [4], Node_Diff:7, Max_Gold_En: 163.433, Energy: 234.389
Source: [5], Node_Diff:7, Max_Gold_En: 163.433, Energy: 233.942
Source: [6], Node_Diff:7, Max_Gold_En: 163.433, Energy: 234.389
Source: [7], Node_Diff:7, Max_Gold_En: 163.433, Energy: 233.942
Source: [8], Node_Diff:7, Max_Gold_En: 163.433, Energy: 237.666
Source: [9], Node_Diff:7, Max_Gold_En: 163.433, Energy: 231.403
Source: [10], Node_Diff:7, Max_Gold_En: 163.433, Energy: 240.281
Source: [11], Node_Diff:7, Max_Gold_En: 163.433, Energy: 240.281
Source: [12], Node_Diff:7, Max_Gold_En: 163.433, Energy: 240.313
Source: [13], Node_Diff:7, Max_Gold_En: 163.433, Energy: 240.324
Source: [14], Node_Diff:7, Max_Gold_En: 163.433, Energy: 238.031
Source: [15], Node_Diff:7, Max_Gold

Source: [0], Node_Diff:7, Max_Gold_En: 163.737, Energy: 235.214
Source: [1], Node_Diff:7, Max_Gold_En: 163.737, Energy: 235.088
Source: [2], Node_Diff:7, Max_Gold_En: 163.737, Energy: 232.341
Source: [3], Node_Diff:7, Max_Gold_En: 163.737, Energy: 233.682
Source: [4], Node_Diff:7, Max_Gold_En: 163.737, Energy: 235.214
Source: [5], Node_Diff:7, Max_Gold_En: 163.737, Energy: 234.788
Source: [6], Node_Diff:7, Max_Gold_En: 163.737, Energy: 235.214
Source: [7], Node_Diff:7, Max_Gold_En: 163.737, Energy: 234.788
Source: [8], Node_Diff:7, Max_Gold_En: 163.737, Energy: 238.450
Source: [9], Node_Diff:7, Max_Gold_En: 163.737, Energy: 232.273
Source: [10], Node_Diff:7, Max_Gold_En: 163.737, Energy: 241.062
Source: [11], Node_Diff:7, Max_Gold_En: 163.737, Energy: 241.066
Source: [12], Node_Diff:7, Max_Gold_En: 163.737, Energy: 241.098
Source: [13], Node_Diff:7, Max_Gold_En: 163.737, Energy: 241.109
Source: [14], Node_Diff:7, Max_Gold_En: 163.737, Energy: 238.816
Source: [15], Node_Diff:7, Max_Gold

In [64]:
train = train_generator(loaded_SKT, loaded_DCS, bz2_input_folder, n_trainset = -1, filePerBatch = 15, iterationPerBatch = 10, _debug=False)

In [20]:
# Complete Training
# tips: try increasing iterations per batch
trainingStatus = defaultdict(lambda: bool(False)) # Reset it after 3 epochs of full-training set
train.__next__()

Weights Saved:  outputs/train_t1993934581183/nnet_i0.p
Batch:  0
['10250.ds.bz2', '428839.ds.bz2', '20035.ds.bz2', '32130.ds.bz2', '418916.ds.bz2', '154658.ds.bz2', '65093.ds.bz2', '266079.ds.bz2', '429657.ds.bz2', '389979.ds.bz2', '365964.ds.bz2', '29623.ds.bz2', '162846.ds.bz2', '296029.ds.bz2', '338298.ds.bz2']
ITERATION IN 0
ITERATION IN 1
ITERATION IN 2
ITERATION IN 3
ITERATION IN 4
ITERATION IN 5
ITERATION IN 6
ITERATION IN 7
Avg. time taken by 1 file(1 iteration): 0.320
Weights Saved:  outputs/train_t1993934581183/nnet.p
Batch:  1
['187738.ds.bz2', '171533.ds.bz2', '310155.ds.bz2', '22207.ds.bz2', '12964.ds.bz2', '231593.ds.bz2', '399906.ds.bz2', '194861.ds.bz2', '130103.ds.bz2', '433682.ds.bz2', '164353.ds.bz2', '35530.ds.bz2', '240430.ds.bz2', '37993.ds.bz2', '57802.ds.bz2']
ITERATION IN 0
ITERATION IN 1
ITERATION IN 2
ITERATION IN 3
ITERATION IN 4
ITERATION IN 5
ITERATION IN 6
ITERATION IN 7
Avg. time taken by 1 file(1 iteration): 0.522
Weights Saved:  outputs/train_t19939345

### Checking bz2 Files

In [118]:
# bz2_input_folder2 = '../NewData/skt_dcs_DS.bz2_1L_bigram_10K/'
dsbz2_name = '226964' + '.ds.bz2'
(nodelist_correct2, conflicts_Dict_correct, featVMat_correct, nodelist_to_correct_mapping,\
    nodelist, conflicts_Dict2, featVMat) = open_dsbz2(bz2_input_folder + dsbz2_name)

In [119]:
nodelist_correct2

[WD_Node[C: 0, P: 0, brU @(-43) => abravIt],
 WD_Node[C: 1, P: 0, rAvaRa @(69) => rAvaRam],
 WD_Node[C: 2, P: 0, sUta @(29) => sUtas],
 WD_Node[C: 3, P: 0, hita @(71) => hitam],
 WD_Node[C: 4, P: 1, anunaya @(71) => anunayam],
 WD_Node[C: 5, P: 0, vacas @(31) => vacas],
 WD_Node[C: 5, P: 0, vacas @(71) => vacas]]

In [120]:
kx = 0
for node in nodelist:
    print(kx, node)
    kx += 1

0 WD_Node[C: 0, P: 0, brU @(-43) => abravIt]
1 WD_Node[C: 1, P: 0, rAvaRa @(69) => rAvaRam]
2 WD_Node[C: 2, P: 0, sUta @(29) => sUtas]
3 WD_Node[C: 2, P: 0, sUta @(29) => sUtas]
4 WD_Node[C: 2, P: 0, sU @(-190) => sUtas]
5 WD_Node[C: 3, P: 0, hi @(2) => hi]
6 WD_Node[C: 3, P: 0, hita @(71) => hitam]
7 WD_Node[C: 3, P: 0, DA @(-190) => hitam]
8 WD_Node[C: 3, P: 2, tad @(69) => tam]
9 WD_Node[C: 3, P: 0, hita @(71) => hitam]
10 WD_Node[C: 4, P: 2, nu @(2) => nu]
11 WD_Node[C: 4, P: 0, tad @(30) => sA]
12 WD_Node[C: 4, P: 0, sa @(3) => sa]
13 WD_Node[C: 4, P: 6, yad @(69) => yam]
14 WD_Node[C: 4, P: 4, naya @(2) => nayam]
15 WD_Node[C: 4, P: 1, anu @(3) => anu]
16 WD_Node[C: 4, P: 4, na @(2) => na]
17 WD_Node[C: 4, P: 1, anu @(3) => anu]
18 WD_Node[C: 4, P: 1, anunaya @(71) => anunayam]
19 WD_Node[C: 5, P: 0, vacas @(71) => vacas]


In [121]:
nodelist_to_correct_mapping

{0: 0, 1: 1, 2: 2, 3: 6, 4: 18, 5: 20, 6: 19}

In [69]:
bz2_input_folder + dsbz2_name

'/home/rs/15CS91R05/vishnu/Data/skt_dcs_DS.bz2_compat_10k_check_again/407288.ds.bz2'

######   THE END
-----------------------------------------------------------------------